In [ ]:
! pip install selenium

In [ ]:
!pip install undetected_chromedriver

In [1]:
import requests
from bs4 import BeautifulSoup
import random
import time
import pandas as pd 
import numpy as np 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
import undetected_chromedriver as uc

#### URL

Abrir el navegador

In [4]:
Browser = uc.Chrome()

In [5]:
url = 'https://www.idealista.com/inmueble/101980713/'

Abrir el url

In [6]:
Browser.get(url)

Presionar aceptar en el cookie

In [7]:
Browser.find_element('xpath','//*[@id="didomi-notice-agree-button"]').click()

Crear el objeto de parseo

In [8]:
html = Browser.page_source

In [9]:
soup = BeautifulSoup(html, 'html.parser')

Extraer información

**Titulo**

Extracción por etiquetas

In [10]:
titulo = soup.find("span",{'class':"main-info__title-main"}).text
titulo

'Piso en venta en calle de José Ortega y Gasset'

**Localizacion**

In [11]:
localizacion = soup	.find('span',{'class':'main-info__title-minor'}).text.split()
localizacion = localizacion[0].replace(',','')
localizacion

'Castellana'

**Precio**

In [12]:
precio = soup.find('span',{'class':'info-data-price'}).text.split()
precio = precio[0].replace('.','')
precio

'7900000'

**Caracteristicas**

In [13]:
car = soup.find('div',{'class':'details-property_features'}).find('ul')
car_basicos = []
for caract in car.find_all('li'):
    car_basicos.append(caract.text)
car_basicos

['473 m² construidos',
 '4 habitaciones',
 '5 baños',
 'Plaza de garaje incluida en el precio',
 'Segunda mano/buen estado',
 'Trastero',
 'Construido en 1900']

**Ubicacion**

In [14]:
ubicacion = soup.find('div',{'id':'headerMap'}).find('ul')
ubi = []

for ubica in ubicacion.find_all('li'):
    ubi.append(ubica.text.strip())
ubi

['Calle de José Ortega y Gasset',
 'Barrio Castellana',
 'Distrito Barrio de Salamanca',
 'Madrid',
 'Madrid capital, Madrid']

### Automatizacion para varios pisos

In [15]:
big_url = 'https://www.idealista.com/venta-viviendas/madrid-madrid/'

In [16]:
Browser = uc.Chrome()

In [ ]:
Browser.get(big_url)

In [ ]:
Browser.find_element('xpath','//*[@id="didomi-notice-agree-button"]').click()

Preparar el parseo

In [ ]:
html = Browser.page_source

In [ ]:
soup = BeautifulSoup(html,'html.parser')

In [ ]:
articulo =  soup.find('main',{'id':'main-content'}).find_all('article')
articulo

In [ ]:
id_pisos = []

for i in articulo:
    if i.get('data-element-id') is not None:
        id_pisos.append(i.get('data-element-id'))

In [ ]:
id_pisos

In [ ]:
len(id_pisos)

**Iterar sobre paginas**

Extraer todos los id de las primeras paginas 

In [2]:
# Seleccionar provincia donde se desea 
ubi_pisos = 'centro'
id_pisos_ubi = []
x = 1
while True:
    
    url = f'https://www.idealista.com/venta-viviendas/madrid/{ubi_pisos}/pagina-{x}.htm'
    Browser = uc.Chrome()
    Browser.get(url)
    try:
        WebDriverWait(Browser,5)
        Browser.find_element('xpath','//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = Browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

    pagina_actual = int(soup.find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)

    if x == pagina_actual:
        articulo =  soup.find('main',{'id':'main-content'}).find_all('article')
    if x == 2:
        break
        
    x = x+1
    
    for i in articulo:
        if i.get('data-element-id') is not None:
            id_pisos_ubi.append(i.get('data-element-id'))
            time.sleep(random.randint(1,5))
    print(id_pisos_ubi)

['101184519', '102711209', '104922553', '104011543', '103933185', '104302587', '103870808', '103813031', '104496367', '104007120', '103042820', '104384209', '104595306', '104203971', '99003851', '104800196', '104696022', '102665714', '102743423', '101514342', '104799107', '104967261', '104606417', '104657376', '104787046', '98972733', '104567397', '102337728', '103199813', '104567294']


In [3]:
len(id_pisos_ubi)

30

**Extraccion de informacion**

In [3]:
df = pd.DataFrame(columns=(['Titulo','Localización','Precio','Caracteristicas','Ubicación']))
datos = []
x = 0
Browser = uc.Chrome()

while x < len(id_pisos_ubi):

    i = id_pisos_ubi[x]
    url_id = f'https://www.idealista.com/inmueble/{i}/'
    Browser.get(url_id)
    try:
        Browser.find_element('xpath','//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    html = Browser.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        print('ID casa:',i)
        titulo = soup.find("span",{'class':"main-info__title-main"}).text
        datos.append(titulo)
        print('titulo publicacion:', titulo)
    except:
        print('')

    try:
        localizacion = soup	.find('span',{'class':'main-info__title-minor'}).text.split()
        localizacion = localizacion[0].replace(',','')
        datos.append(localizacion)
        print('localizacion:', localizacion)
    except:
        print('')

    try:
        precio = soup.find('span',{'class':'info-data-price'}).text.split()
        precio = precio[0].replace('.','')
        datos.append(precio)
        print('precio:', precio)
    except:
        print('')
        
    try:
        car = soup.find('div',{'class':'details-property_features'}).find('ul')
        car_basicos = []
        for caract in car.find_all('li'):
            basicos = caract.text
            car_basicos.append(caract.text)
        datos.append(car_basicos)
        print('carcateristicas:',car_basicos)
    except:
        car_basicos = []
        
    try:
        ubicacion = soup.find('div',{'id':'headerMap'}).find('ul')
        ubi = []
        for ubica in ubicacion.find_all('li'):
            ubication = ubica.text.strip()
            ubi.append(ubica.text.strip())
        datos.append(ubi)
        print('Ubicacion:',ubi)
    except:
        ubi = []

    df = df._append({'Titulo':titulo,'Localización':localizacion,'Precio':precio,'Caracteristicas':car_basicos,'Ubicación':ubi}, ignore_index=True)
    print('')
    x = x + 1
Browser.close()
Browser.quit()

ID casa: 101184519
titulo publicacion: Piso en venta en calle Mayor
localizacion: Sol
precio: 752248
carcateristicas: ['132 m² construidos', '3 habitaciones', '2 baños', 'Balcón', 'Segunda mano/para reformar', 'Construido en 1880', 'Calefacción individual: Eléctrica']
Ubicacion: ['Calle Mayor', 'Barrio Sol', 'Distrito Centro', 'Madrid', 'Madrid capital, Madrid']

ID casa: 102711209
titulo publicacion: Piso en venta en Carrera de San Jerónimo
localizacion: Sol
precio: 2350000
carcateristicas: ['218 m² construidos', '4 habitaciones', '4 baños', 'Terraza', 'Segunda mano/buen estado', 'Construido en 1920', 'Calefacción individual: Gas natural']
Ubicacion: ['Carrera de San Jerónimo', 'Barrio Sol', 'Distrito Centro', 'Madrid', 'Madrid capital, Madrid']

ID casa: 104922553
titulo publicacion: Ático en venta en Huertas-Cortes
localizacion: Centro
precio: 890000
carcateristicas: ['84 m² construidos', '2 habitaciones', '1 baño', 'Segunda mano/buen estado', 'Armarios empotrados', 'Orientación est

In [4]:
datos

['Piso en venta en calle Mayor',
 'Sol',
 '752248',
 ['132 m² construidos',
  '3 habitaciones',
  '2 baños',
  'Balcón',
  'Segunda mano/para reformar',
  'Construido en 1880',
  'Calefacción individual: Eléctrica'],
 ['Calle Mayor',
  'Barrio Sol',
  'Distrito Centro',
  'Madrid',
  'Madrid capital, Madrid'],
 'Piso en venta en Carrera de San Jerónimo',
 'Sol',
 '2350000',
 ['218 m² construidos',
  '4 habitaciones',
  '4 baños',
  'Terraza',
  'Segunda mano/buen estado',
  'Construido en 1920',
  'Calefacción individual: Gas natural'],
 ['Carrera de San Jerónimo',
  'Barrio Sol',
  'Distrito Centro',
  'Madrid',
  'Madrid capital, Madrid'],
 'Ático en venta en Huertas-Cortes',
 'Centro',
 '890000',
 ['84 m² construidos',
  '2 habitaciones',
  '1 baño',
  'Segunda mano/buen estado',
  'Armarios empotrados',
  'Orientación este',
  'Construido en 1900',
  'Calefacción individual: Gas natural'],
 ['Barrio Huertas-Cortes',
  'Distrito Centro',
  'Madrid',
  'Madrid capital, Madrid'],
 'Pi

In [6]:
len(datos)

150

In [7]:
df

,Titulo,Localización,Precio,Caracteristicas,Ubicación
0,Piso en venta en calle Mayor,Sol,752248,"[132 m² construidos, 3 habitaciones, 2 baños, ...","[Calle Mayor, Barrio Sol, Distrito Centro, Mad..."
1,Piso en venta en Carrera de San Jerónimo,Sol,2350000,"[218 m² construidos, 4 habitaciones, 4 baños, ...","[Carrera de San Jerónimo, Barrio Sol, Distrito..."
2,Ático en venta en Huertas-Cortes,Centro,890000,"[84 m² construidos, 2 habitaciones, 1 baño, Se...","[Barrio Huertas-Cortes, Distrito Centro, Madri..."
3,Piso en venta en calle de Augusto Figueroa,Chueca-Justicia,1150000,"[150 m² construidos, 2 habitaciones, 2 baños, ...","[Calle de Augusto Figueroa, Barrio Chueca-Just..."
4,Piso en venta en Sol,Centro,900000,"[173 m² construidos, 4 habitaciones, 2 baños, ...","[Barrio Sol, Distrito Centro, Madrid, Madrid c..."
5,Piso en venta en Chueca-Justicia,Centro,1850000,"[150 m² construidos, 3 habitaciones, 3 baños, ...","[Barrio Chueca-Justicia, Distrito Centro, Madr..."
6,Piso en venta en Chueca-Justicia,Centro,1690000,"[145 m² construidos, 3 habitaciones, 2 baños, ...","[Barrio Chueca-Justicia, Distrito Centro, Madr..."
7,Piso en venta en Chueca-Justicia,Centro,850000,"[118 m² construidos, 3 habitaciones, 3 baños, ...","[Barrio Chueca-Justicia, Distrito Centro, Madr..."
8,Piso en venta en Chueca-Justicia,Centro,1850000,"[148 m² construidos, 3 habitaciones, 3 baños, ...","[Barrio Chueca-Justicia, Distrito Centro, Madr..."
9,Piso en venta en Malasaña-Universidad,Centro,730000,"[104 m² construidos, 3 habitaciones, 2 baños, ...","[Barrio Malasaña-Universidad, Distrito Centro,..."


In [8]:
df.shape

(30, 5)

In [9]:
df.iloc[0,3:4]

Caracteristicas    [132 m² construidos, 3 habitaciones, 2 baños, ...
Name: 0, dtype: object

**Guardar con extensión csv**

In [10]:
df.to_csv('Pisos_centro_madrid.csv')